# Load libraries and functions

In [118]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [119]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

## Main Code

In [120]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# # occupation = 'shampooers'
# # occupation = 'dredgeOperators'


GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [121]:
# Load the data
onet = pd.read_csv(f'{data_path}/data/onet_occupations_yearly.csv')
onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
onet = onet[onet['year'] == 2023].reset_index(drop=True)

# Get list of tasks
my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
tasks = my_df['task'].unique().tolist()
tasks

['Decline excessive risks.',
 'Write to field representatives, medical personnel, or others to obtain further information, quote rates, or explain company underwriting policies.',
 'Evaluate possibility of losses due to catastrophe or excessive insurance.',
 'Decrease value of policy when risk is substandard and specify applicable endorsements or apply rating to ensure safe, profitable distribution of risks, using reference materials.',
 'Review company records to determine amount of insurance in force on single risk or group of closely related risks.',
 'Authorize reinsurance of policy when risk is high.',
 'Examine documents to determine degree of risk from factors such as applicant health, financial standing and value, and condition of property.']

<br>

<br>

# 3) GPT First Last Task Method

In [122]:
version = 'v0/'
version = 'v1/'

In [123]:
# Set up output file names
firstLastTask_output_filename = f'{occupation_folder}/{version}{occupation}_firstLastTask.csv'
firstLastTask_DAG_output_filename = f'{occupation_folder}/{version}{occupation}_firstLastTaskGPT_DAG_df.csv'

### Set up the questions for GPT

In [124]:
if version == 'v0/':
    task_relationships_question_options = {'A first': "A would be done first",
                                          'B first': "B would be done first",
                                          'neither': "These are not part of the same task sequence"
                                          }
    firstLastTask_question_text = dedent("""\
                                     Consider {{ occupation }} as an occupation.
                                     The first task (or set of tasks) to be completed for the job is: {{ first_task }}.
                                     The last task (or set of tasks) to be completed for the job is: {{ last_task }}. 
                                     Now consider these two tasks:
                                     A) {{ task_A }} 
                                     B) {{ task_B }}
                                     What is the relationship between these tasks?
                                     Avoid using words like "task A" and "task B" in the answer.
                                     Explain the reasoning behind your answer in a couple of sentences.
                                     """)

In [125]:
if version == 'v1/':
    task_relationships_question_options = {'A first': "Worker working on task B needs to know the output of worker working ong task A",
                                          'B first': "Worker working on task A needs to know the output of worker working ong task B",
                                          'neither': "Neither worker needs to know the output of the other worker"
                                          }
    firstLastTask_question_text = dedent("""\
                                     Consider {{ occupation }} as an occupation.
                                     The first task (or set of tasks) to be completed for the job is: {{ first_task }}.
                                     The last task (or set of tasks) to be completed for the job is: {{ last_task }}. 
                                     Now consider these two tasks:
                                     A) {{ task_A }} 
                                     B) {{ task_B }}
                                     Imagine there are two workers, one working on task A and the other on task B.
                                     Does the worker working on task B need to know the output of the worker working on task A before getting started? What about the opposite?
                                     Avoid using words like "task A" and "task B" in the answer.
                                     Explain the reasoning behind your answer in a couple of sentences.
                                     """)

In [126]:
task_relationships_question_options_list = list(task_relationships_question_options.values())

In [127]:
def task_relationships_firstLast_included(occupation, tasks, first_task, last_task, question_text, question_options):
    if '"Sink"' in tasks:
        tasks.remove('"Sink"')

    # Modify the first task and last task to appear as a single string
    first_task = " And ".join(first_task)
    last_task = " And ".join(last_task)

    scenarios = [Scenario({"occupation": occupation, 
                           "task_A": task_A, "task_B": task_B,
                           "first_task": first_task, "last_task": last_task}) 
                           for task_A, task_B in combinations(tasks, 2)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results


### Use One Step Method: Directly ask for pairwise comparison w/o giving the "either" option
### Next determine first and last task/tasks to be done in the sequence and ask GPT to produce DAG

In [128]:
def first_last_tasks(occupation, tasks):
    # Remove "Sink" node if it exists
    if '"Sink"' in tasks:
        tasks.remove('"Sink"')

    scenarios = [Scenario({"occupation": GPT_input_occupation, "tasks": tasks})]

    # First task
    q1 = QuestionCheckBox(
        question_name = "firstTask",
        question_text = dedent("""\
            Consider {{ occupation }} as an occupation.
            The tasks below are part of the job of a {{ occupation }}: {{ tasks }}.
            Among the following, which task or set of tasks would be done before all other tasks in order to compelete the job?
            """),
        question_options = tasks,
        min_selections = 1,
        max_selections = 3
    )
    results1 = q1.by(m4).by(scenarios).run().to_pandas()
    first_task = results1['answer.firstTask'][0]
    first_task = ast.literal_eval(first_task) # convert from string resembling list format to actual list


    # Last task
    q2 = QuestionCheckBox(
        question_name = "lastTask",
        question_text = dedent("""\
            Consider {{ occupation }} as an occupation. 
            The tasks below are part of the job of {{ occupation }}: {{ tasks }}.
            Among the following, which task or set of tasks would be done after all other tasks are completed?
            """),
        question_options = tasks,
        min_selections = 1,
        max_selections = 3
    )
    results2 = q2.by(m4).by(scenarios).run().to_pandas()
    last_task = results2['answer.lastTask'][0]
    last_task = ast.literal_eval(last_task) # convert from string resembling list format to actual list
    
    return first_task, last_task

In [129]:
first_task, last_task = first_last_tasks(GPT_input_occupation, tasks)
firstLast_tasks_df = pd.DataFrame({'first_task': [first_task], 'last_task': [last_task]})
firstLast_tasks_df.to_csv(firstLastTask_output_filename, index=False)

### Compare pair of tasks

In [130]:
results = task_relationships_firstLast_included(GPT_input_occupation, tasks, first_task, last_task, firstLastTask_question_text, task_relationships_question_options_list)
#results.select("task_A", "task_B", "ordering", "comment.ordering_comment").print()
GPT_firstLast_df_raw = results.select("task_A", "task_B", "ordering", "comment.ordering_comment").to_pandas()

# Swap columns and subset only those that are part of the same task sequence 
GPT_firstLast_df = GPT_firstLast_df_raw.copy()
mask = GPT_firstLast_df['answer.ordering'] == task_relationships_question_options['B first']
GPT_firstLast_df.loc[mask, ['scenario.task_A', 'scenario.task_B']] = GPT_firstLast_df.loc[mask, ['scenario.task_B', 'scenario.task_A']].values
GPT_firstLast_df.loc[mask, 'answer.ordering'] = task_relationships_question_options['A first']
GPT_firstLast_df = GPT_firstLast_df[GPT_firstLast_df['answer.ordering'] == task_relationships_question_options['A first']]
GPT_firstLast_df = GPT_firstLast_df[['scenario.task_A', 'scenario.task_B', 'comment.ordering_comment']]

# Change column names
GPT_firstLast_df = GPT_firstLast_df.rename(columns={'scenario.task_A': 'source', 
                                                    'scenario.task_B': 'target', 
                                                    'comment.ordering_comment': 'comment'})

# Save output
GPT_firstLast_df.to_csv(firstLastTask_DAG_output_filename, index=False)

Output()